In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Embedding,Flatten,LeakyReLU,BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from glob import glob

In [3]:
print(tf.__version__)

2.1.0


In [4]:
IMAGE_SIZE=[224,224]
train_image='D:\Dataset\Medicines\Medicine\Train Data'

In [5]:
folder=glob('D:\Dataset\Medicines\Medicine\Train Data\*')

In [6]:
len(folder)

4

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [8]:
training_set=train_datagen.flow_from_directory('D:\Dataset\Medicines\Medicine\Train Data',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)


Found 80 images belonging to 4 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:\Dataset\Medicines\Medicine\Test Data',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 20 images belonging to 4 classes.


# Transfer Learning techq

# 1) InceptionV3

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [11]:
inception=tf.keras.applications.InceptionV3(input_shape=IMAGE_SIZE + [3],weights="imagenet",include_top=False)

In [12]:
for layers in inception.layers:
    layers.trainable=False

In [13]:
flat=Flatten()(inception.output)

In [14]:
predictions=Dense(len(folder),activation="softmax")(flat)

In [15]:
model_inception=Model(inputs=inception.inputs,outputs=predictions)

In [16]:
model_inception.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [17]:
model_inception.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

In [18]:
model_1=model_inception.fit_generator(training_set,epochs=19,steps_per_epoch=len(training_set))

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 16 steps
Epoch 1/19
16/16 [==============================] - 10s 631ms/step - loss: 5.0292 - accuracy: 0.3750
Epoch 2/19
16/16 [==============================] - 6s 376ms/step - loss: 1.8849 - accuracy: 0.8000
Epoch 3/19
16/16 [==============================] - 6s 379ms/step - loss: 0.5305 - accuracy: 0.8750
Epoch 4/19
16/16 [==============================] - 6s 381ms/step - loss: 0.0679 - accuracy: 0.9750
Epoch 5/19
16/16 [==============================] - 5s 337ms/step - loss: 0.1516 - accuracy: 0.9625
Epoch 6/19
16/16 [==============================] - 6s 371ms/step - loss: 0.0362 - accuracy: 0.9875
Epoch 7/19
16/16 [==============================] - 6s 352ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 8/19
16/16 [==============================] - 6s 345ms/step - loss: 0.0473 - accuracy: 0.9875
Epoch 9/19
16/16 [==============================] - 6s 344ms/step - loss: 0.086

In [20]:
from tensorflow.keras.models import load_model

model_inception.save("model_inception.h5")



In [225]:
model_inception=load_model('model_inception.h5')

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model


image1=image.load_img('D:/Dataset/Medicines/Medicine/Train Data/Statin/5.jpg',target_size=(224,224))



In [226]:
from tensorflow.keras.preprocessing import image
output=image.img_to_array(image1)
print(output)

[[[190. 142. 142.]
  [190. 142. 142.]
  [191. 143. 143.]
  ...
  [154. 109. 106.]
  [153. 105. 103.]
  [152. 104. 102.]]

 [[190. 142. 142.]
  [190. 142. 142.]
  [191. 143. 143.]
  ...
  [154. 109. 106.]
  [153. 105. 103.]
  [152. 104. 102.]]

 [[190. 142. 142.]
  [190. 142. 142.]
  [191. 143. 143.]
  ...
  [154. 109. 106.]
  [153. 105. 103.]
  [152. 104. 102.]]

 ...

 [[173. 141. 146.]
  [175. 143. 148.]
  [176. 144. 149.]
  ...
  [172. 149. 159.]
  [179. 153. 164.]
  [178. 152. 163.]]

 [[173. 141. 146.]
  [175. 143. 148.]
  [176. 144. 149.]
  ...
  [172. 149. 159.]
  [179. 153. 164.]
  [178. 152. 163.]]

 [[173. 141. 146.]
  [175. 143. 148.]
  [176. 144. 149.]
  ...
  [174. 151. 161.]
  [179. 153. 164.]
  [178. 152. 163.]]]


In [227]:
print(output.shape)

(224, 224, 3)


In [228]:
output/255
output=np.expand_dims(output,axis=0)
img_output=preprocess_input(output)
print(img_output.shape)

(1, 224, 224, 3)


In [229]:
model_inception.predict(img_output)
a=np.argmax(model_inception.predict(img_output), axis=1)

In [230]:
print(a)

[3]
